<a href="https://colab.research.google.com/github/alexweltgeist/twitter_bitcoin/blob/main/Create_Trainingset_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create Trainingset for CNN**

Download Twetts, apply sentiment via Textblob and lable the tweets

In [25]:
!pip install twython    
!pip install textblob
!pip install tweet-preprocessor

In [10]:
from twython import TwythonStreamer
from twython import Twython
import pandas as pd
from textblob import TextBlob
import preprocessor as p

In [11]:
# My key and secret
CONSUMER_KEY = 'XP9WEtNESINs3LfjkyFYLIN8l'
CONSUMER_SECRET = 'VeVcYCxyGNJHAq5mwzJZXiAfioXeAajhrcQBYcU1uDjPShqmCS'
ACCESS_TOKEN = '1126001584830390272-JnhR5hrvd0YX9tnIpBTj3uNkhx3tsW'
ACCESS_TOKEN_SECRET = 'yLXT7Tcsr0Xx7T7ldUfyHs8Eltrro18gVWnBm0Xo2Q54m'

In [12]:
# Functions for Streaming
del(tweets, twitter_df)
tweets = [] 

class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        """
        data in Python dict representing a tweet
        """
        # We only want to collect English-language tweets
        if data.get('lang') == 'en':
            tweets.append(data)
            if (len(tweets) % 10) == 0:
              print(f"received tweet #{len(tweets)}")

		# Stop when we've collected enough
        if len(tweets) >= 1000:
            self.disconnect()
        return tweets

    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()


In [16]:
# Functions for data extraction
def tcontent(stream):
    # Preprocessing options used for p.clean() below
    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY)
    textList = []
    for tweet in tweets:   
        textList.append(p.clean(tweet['text']))
    return textList

def sentiment(stream):
    dateList = []
    for tweet in tweets:   
        dateList.append(TextBlob(tweet['text']).sentiment.polarity)
    return dateList

def tnames(stream):
    nameList = []
    for tweet in tweets:   
        nameList.append(tweet['user']['name'])
    return nameList

def tdate(stream):
    dateList = []
    for tweet in tweets:   
        dateList.append(tweet['created_at'])
    return dateList


# Define the filter criteria
selection = ['Bitcoin', 'bitcoin']


In [ ]:
# Consume public tweets with selection and create pandas dataframe
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
					ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

stream.statuses.filter(track=selection)


In [17]:
twitter_df = pd.concat([pd.Series(tcontent(tweets)), 
                        pd.Series(sentiment(tweets))],
                        axis=1)
twitter_df.columns = ['text', 'sentiment']

# Get rid of Rows with no Polarity:
# twitter_df = twitter_df[twitter_df['sentiment'] != 0]


In [ ]:
twitter_df.head(5)

In [21]:
def create_lable(polarity):
    polarity = float(polarity)
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive"

In [22]:
# Convert from Sentiment to Lable
twitter_df['lable'] = twitter_df.sentiment.apply(create_lable)
twitter_df = twitter_df.drop(['sentiment'], axis=1)


In [23]:
twitter_df.head(5)

,text,lable
0,"RT @ShibInformer: Update: 506,500 $SHIB holder...",Neutral
1,Perfect Brings Makeup Into The Connected Econo...,Positive
2,RT @gate_io: CMO @marietatibouet recently spok...,Neutral
3,RT @maxkeiser: #Bitcoin and the @michael_saylo...,Neutral
4,#Bitcoin Mining Difficulty Falls to January Le...,Neutral


In [24]:
# Save dataframe 
path_to_save = "/content"
twitter_df.to_csv(path_to_save + '/all_tweets_with_lable.csv', sep=",", index=False)                             
